In [1]:
import time
import psutil
from collections import deque


from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Range1d

from nupic.data.inference_shifter import InferenceShifter
from nupic.frameworks.opf.modelfactory import ModelFactory

import model_params

SECONDS_PER_STEP = 2
WINDOW = 60

output_notebook()
fig = figure(title='Memory usage prediction example', y_range=(0,100))
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "Memory usage [%]"
# Keep the last WINDOW predicted and actual values for plotting.
actual_history = deque([0.0] * WINDOW, maxlen=60)
prediction_history = deque([0.0] * WINDOW, maxlen=60)
anomaly_history = deque([0.0] * WINDOW, maxlen=60)

# Initialize the plot lines that we will update with each new record.
actual_line = fig.line(range(WINDOW), actual_history, color='blue', legend='History', line_width=5)
prediction_line = fig.line(range(WINDOW), prediction_history, color='black', legend='Prediction', line_dash="dashed", line_width=3)
anomaly_bar = fig.vbar(range(WINDOW), top=anomaly_history, width=1, bottom=0, color='red', legend='Anomaly likelihood', fill_alpha=0.2)


Loading BokehJS ...

In [2]:
def update_metrics():
  """Poll memory usage, make predictions, and plot the results. Runs forever."""
  # Create the model for predicting CPU usage.
  model = ModelFactory.create(model_params.MODEL_PARAMS)
  model.enableInference({'predictedField': 'memory'})
  # The shifter will align prediction and actual values.
  shifter = InferenceShifter()

  while True:
    s = time.time()

    # Get the memory usage.
    memory = (float(psutil.virtual_memory().used)/float(psutil.virtual_memory().total)) * 100
  
    # Run the input through the model and shift the resulting prediction.
    modelInput = {'memory': memory}
    result = shifter.shift(model.run(modelInput))

    # Update the trailing predicted and actual value deques.
    inference = result.inferences['multiStepBestPredictions'][1]
    anomaly_score = result.inferences['anomalyScore']*100
    if inference is not None:
      actual_history.append(result.rawInput['memory'])
      prediction_history.append(inference)
      anomaly_history.append(anomaly_score)
      actual_line.data_source.data["y"] = actual_history
      prediction_line.data_source.data["y"] = prediction_history
      anomaly_bar.data_source.data["top"] = anomaly_history
      push_notebook(handle=h)
    time.sleep(SECONDS_PER_STEP)


In [3]:
h = show(fig, notebook_handle=True)

In [ ]:
update_metrics()